In [ ]:
import qkit
qkit.start()

from qkit.storage.store import Data

# Probe Station Analysis Notebook
This notebook performs analysis on data gathered by the coresponding measurement notebook.

Cells requiring your input are preceded by **bold text indicating user information.**

First, Qkit needs to load the data gathered from the probe station. **Input the UUID below.**

In [ ]:
h5 = Data(qkit.fid.get('RSYBR9'))

Using numpy, the data is collected into a 1d-Array per measurement.

In [ ]:
# Prepare Data Set
import numpy as np

# Step 1: Accumulate data for each sample
i_collected = np.concatenate([a[:] for a in [h5.data.i_0, h5.data.i_1]], axis = -1)
v_collected = np.concatenate([a[:] for a in [h5.data.v_0, h5.data.v_1]], axis = -1)
dvdi_collected = np.concatenate([a[:] for a in [h5.analysis.dvdi_0, h5.analysis.dvdi_1]], axis = -1)
print("Raw data shapes:", i_collected.shape, v_collected.shape, dvdi_collected.shape)


## Sanity Check: Check for faulty samples
Sanity checks are performed to identify potential errors, such as misconfigured measurements (out of range), open line and short circuits.

Here, a resistance below 20 Ohm is assumed to imply a short circuit. **Adjust the SHORT_CIRCUIT_RESISTANCE value as required for your samples.**

In [ ]:

# Get maximum i and v, naive resistance calculation
i_max = np.max(np.abs(i_collected), axis=-1)
v_max = np.max(np.abs(v_collected), axis=-1)
print("Max shapes:", i_max.shape, v_max.shape)

i_normalized = i_collected / i_max[:, :, np.newaxis]
v_normalized = v_collected / v_max[:, :, np.newaxis]
dvdi_normalized = dvdi_collected / (v_max / i_max)[:, :, np.newaxis]
print("Normalized data shapes:", i_normalized.shape, v_normalized.shape, dvdi_normalized.shape)

# Sanity check:
SHORT_CIRCUIT_RESISTANCE = 20 # Adjust here
excessive_derivatives = np.max(np.abs(dvdi_normalized), axis=-1) > 2
flat_sections = np.min(np.abs(dvdi_normalized), axis=-1) < 0.5
short_circuit = np.max(np.abs(dvdi_collected), axis=-1) < SHORT_CIRCUIT_RESISTANCE
faulty = excessive_derivatives | flat_sections | short_circuit
print("Classification shape", faulty.shape)

An overview over the location of the faulty samples is generated. This can be used to identify scratches or similar mechanical root causes.

In [ ]:
# Show faulty map
import matplotlib.pyplot as plt
plt.imshow(faulty)
plt.show()

All faulty and working samples are plotted in two plots. This serves to verify the classification is working.

In [ ]:
# Display all sample:
import matplotlib.pyplot as plt
# Show faulty
it = np.nditer(faulty, flags=['multi_index'])
for defective in it:
    if defective:
        plt.plot(i_normalized[it.multi_index], v_normalized[it.multi_index])
plt.show()

# Show not faulty
it = np.nditer(faulty, flags=['multi_index'])
for defective in it:
    if not defective:
        plt.plot(i_normalized[it.multi_index], v_normalized[it.multi_index])
plt.show()

# Show not faulty
it = np.nditer(faulty, flags=['multi_index'])
for defective in it:
    if not defective:
        plt.plot(i_normalized[it.multi_index], dvdi_normalized[it.multi_index])
plt.show()

## Visualize Resistance and Conductivity across the sample

In [ ]:
# plot RT resistance across the chip:
from matplotlib.colors import LogNorm
average_resistance = np.average(dvdi_collected, axis=-1)
average_resistance[np.where(faulty)] = 0
non_faulty_data = average_resistance[np.where(np.logical_not(faulty))]

plt.imshow(average_resistance, norm=LogNorm(vmin=np.min(non_faulty_data), vmax=np.max(non_faulty_data)))
plt.colorbar()
plt.title("Resistance")
plt.show()

In [ ]:
from matplotlib.colors import LogNorm
average_conductivity = 1/np.average(dvdi_collected, axis=-1)
average_conductivity[np.where(faulty)] = 0
non_faulty_data = average_conductivity[np.where(np.logical_not(faulty))]

plt.imshow(average_conductivity, norm=LogNorm(vmin=np.min(non_faulty_data), vmax=np.max(non_faulty_data)))
plt.colorbar()
plt.title("Conductivity")
plt.show()

### Visualize conductivity along one axis
Here, it is assumed that all samples along one axis are comparable. **This might need to be change.**

This data can be used for fitting models, to confirm or refute hypotheses about how / if the sample works.

In [ ]:
indices =  np.where(np.logical_not(faulty))
# print(indices.shape, non_faulty_data.shape)
plt.scatter(indices[1], non_faulty_data) # Change axis along which samples are comparable
plt.show()